In [1]:
%%capture
import numpy as np
import pandas as pd

# import warnings
# warnings.filterwarnings("ignore")
from babel.numbers import format_currency
from calitp import to_snakecase

# Display
from IPython.display import HTML, Image, Markdown, display, display_html
from shared_utils import altair_utils, styleguide

# Settings
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

# Charts
import altair as alt
from shared_utils import calitp_color_palette as cp

# GCS, del later since this will presumbly be read from a script that cleans up the data
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"
FILE = "fake_data.xlsx"

# My utilities
import _utils

In [2]:
df = pd.read_excel(f"{GCS_FILE_PATH}{FILE}", sheet_name="fake")

In [3]:
# Parameter Cell
# Other projects: 'Equitable And Sustainable Public Transit Planning For Rural America' in D1
# 'I-880/Decoto Interchange Modernization Project' in D4
# 'Monroe Street Bridge Project' in D8
# 'Forrester Rd Widening And Realignment' in D11
project_name = 'Forrester Rd Widening And Realignment'

In [4]:
# Objects

# DF with project that was set in the parameter cell
df2 = df.loc[df["project_name"] == project_name].reset_index(drop=True)

# Grab district of parameter cell project
district_object = df2["district"].values[0]

# DF with projects in the same district as project
df_district = df.loc[df["district"] == district_object]

# Median benefit score  in district
district_median_benefit_score = int(df_district.fake_benefit_score.median())

# Median project cost in district
district_median_cost = format_currency(
    (df_district["total_project_cost__$1,000_"].median()),
    currency="USD",
)

# Median money requested.
district_median_req = format_currency(
    (df_district["current_fake_fund_requested"].median()),
    currency="USD",
)

In [5]:
# Formatting
# Format some currency columns
df2["formatted_total_project"] = df2["total_project_cost__$1,000_"].apply(
    lambda x: format_currency(x, currency="USD", locale="en_US")
)

df2["formatted_requested"] = df2["current_fake_fund_requested"].apply(
    lambda x: format_currency(x, currency="USD", locale="en_US")
)

In [6]:
# Fake title
display(
    Markdown(
        f"""<h1><b>{df2.project_name.values[0]}</b></h1>
        <br> This project (PPNO:{df2.ppno.values[0]}) is led by <b>{df2.lead_agency.values[0]}</b> in <b>District {df2.district.values[0]}</b> and 
        is in its <b>{df2.current_phase[0]}</b> phase.
        <br><br><b>Project Description:</b> {df2['project_description'].values[0]}
        """
    )
)

<h1><b>Forrester Rd Widening And Realignment</b></h1>
        <br> This project (PPNO:1217) is led by <b>None</b> in <b>District 11</b> and 
        is in its <b>PID</b> phase.
        <br><br><b>Project Description:</b> Imperial County’s 2013 Transportation Plan lists this project in its Mid-term Constrained Plan and proposes to build Forrester Road as a four-lane expressway from I-8 to the SR 78 intersection in the City of Westmorland. This project will improve the transportation of goods and people along Forrester Road by upgrading the existing roadway to one that can handle the anticipated increase in traffic.  The project also includes a truck by-pass to the City of Westmorland. 

This project addresses the use of Forrester Road as a bypass to the Cities of El Centro, Imperial, and Brawley. It is also the preferred route to take the place of the relinquished portion of SR 86.
Forrester Road is currently being used to bypass the Cities of El Centro, Imperial, Brawley, and Westmorland. It provides a travel times savings between the southern portion of Imperial County and the Indio/Palm Springs area. Within the Cities of El Centro, Brawley, and Westmorland, SR 86 operates more like a city arterial and serves as a Main Street with a high concentration of access points including pedestrian crossings. The State of California has already relinquished to the City of Imperial the portion of SR 86 within their city limits. The long-term plan is to relinquish all of SR 86 between SR 111 (at Heber Road) and the Fredericks Road/SR 78 Junction to the local agencies.
        

In [7]:
display(
    Markdown(
        f"""<h2>Basic Project Statistics</h2>
        <li>Project Category: <b>{df2['primary_mode'].values[0]}</b></li>
        <li>Benefit Score: <b>{df2['fake_benefit_score'].values[0].astype('int64')}</b></li>
        <li>Statewide Benefit Rank: <b>{df2['statewide_rank'].values[0]}</b> out of {len(df)}</li>
        <li>District Benefit Rank: <b>{df2['district_rank'].values[0]}</b> out of {len(df.loc[df["district"]==district_object])}</li>
        <br><b>Important Dates</b><br>
        <li>PID Approval: <b>{df['pid_approval_date'].values[0]}</b>
        <li>PA/ED: <b>{df['target_pa_ed'].values[0]}</b>
        <li>RTL: <b>{df['rtl_date'].values[0]}</b>
        <li>Construction: <b>{df['con_start_date'].values[0]}</b>
        """
    )
)

<h2>Basic Project Statistics</h2>
        <li>Project Category: <b>Highway</b></li>
        <li>Benefit Score: <b>7</b></li>
        <li>Statewide Benefit Rank: <b>469</b> out of 717</li>
        <li>District Benefit Rank: <b>22</b> out of 35</li>
        <br><b>Important Dates</b><br>
        <li>PID Approval: <b>datetime64[ns]</b>
        <li>PA/ED: <b>None</b>
        <li>RTL: <b>datetime64[ns]</b>
        <li>Construction: <b>datetime64[ns]</b>
        

In [8]:
# Calculate funded %
df2["funded"] = (
    df2["current_fake_fund_requested"] / df2["total_project_cost__$1,000_"]
)


# Remaining project cost
df2["unfunded"] = (1-df2["funded"]
)

# Format
df2["percentage_req_formatted"] = (
    df2["funded"].astype(float).map("{:.0%}".format)
)

df2["unfunded_percentage_formatted"] = (
    df2["unfunded"].astype(float).map("{:.0%}".format)
)



In [9]:
#  In comparison,the median total cost of projects in {df2.district[0]} is {district_median_cost} and the median amount requested is {district_median_req}. 

In [10]:
display(
    Markdown(
        f"""<h2>Funding</h2>
        {df2.project_name[0]} costs <b>{df2.formatted_total_project[0]}</b> in total.
        They have requested <b>{df2.formatted_requested[0]}</b>, which covers about <b>{df2.percentage_req_formatted[0]}</b> of the project's cost.
        <br> This project could potentially qualify for additional fund(s) from {df2.potential_funding_program_s_[0]}.
        
        """
    )
)

<h2>Funding</h2>
        Forrester Rd Widening And Realignment costs <b>$888,888.00</b> in total.
        They have requested <b>$367,809.57</b>, which covers about <b>41%</b> of the project's cost.
        <br> This project could potentially qualify for additional fund(s) from Tcep.
        
        

In [11]:
# Create a new dataframe for percentage funded

# Transform
percentage = df2.T.reset_index().rename(columns={"index": "Percentage", 0: "Value"})

# Select only certain values
percentage = percentage[
    percentage["Percentage"].isin(
        [
            "unfunded",
            "funded",
            
        ]
    )
]

# Remove underscores off of old column names
percentage["Percentage"] = (
    percentage["Percentage"].str.replace("_", " ").str.title()
)

# Reset Index
percentage = percentage.reset_index(drop=True)

In [12]:
funding_pie_chart = _utils.basic_pie_chart(percentage,  "Value:Q", "Percentage:N", "Percentage:N",
                  "Percentage Funded vs. Unfunded")

In [13]:
# Create a new dataframe for funding

# Transform
funding = df2.T.reset_index().rename(columns={"index": "Monetary Columns", 0: "Value"})

# Select only certain values
funding = funding[
    funding["Monetary Columns"].isin(
        [
            "total_project_cost__$1,000_",
            "current_fake_fund_requested",
        ]
    )
]

# Remove underscores off of old column names
funding["Monetary Columns"] = (
    funding["Monetary Columns"].str.replace("_", " ").str.title()
)

# Add a column with formatted values
funding["Total"] = funding["Value"].apply(
    lambda x: format_currency(x, currency="USD", locale="en_US")
)

# Reset Index
funding = funding.reset_index(drop=True)

In [14]:
requested_bar_chart = _utils.basic_bar_chart_custom_tooltip(
    funding,
    "Value",
    "Monetary Columns",
    "Total",
    "Monetary Columns",
    chart_title="Total Project Cost versus Requested",
)

In [15]:
requested_bar_chart | funding_pie_chart

alt.HConcatChart(...)

In [16]:
display(
    Markdown(
        f"""<h2>Score Card</h2>
        The total score is <b>{(df2.fake_benefit_score[0]).astype('int64')}</b>, compared to the median score of 
        <b>{district_median_benefit_score}</b> of other projects in District {df2.district[0]}. 
        <br>Click on the bar for more detail.
        """
    )
)

<h2>Score Card</h2>
        The total score is <b>7</b>, compared to the median score of 
        <b>8</b> of other projects in District 11. 
        <br>Click on the bar for more detail.
        

In [17]:
# df2.drop(columns=["project_description"]).head()

In [18]:
# Create new df for score card
score_card = df2[
    [
        "increase_peak_person_throughput",
        "reduction_in_peak_period_delay",
        "reduction_in_fatal_and_injury_crashes",
        "reduction_in_injury_rates",
        "increase_access_to_jobs",
        "increase_access_jobs_to_DAC",
        "commercial_dev_developed",
        "tons_of_goods_impacted",
        "improve_air_quality",
        "impact_natural_resources",
        "support_of_trasnportation",
    ]
]

In [19]:
# Transform
score_card = score_card.T.reset_index().rename(
    columns={"index": "Measure", 0: "Measure Score"}
)
# Remove underscores off of old column names
score_card["Measure"] = score_card["Measure"].str.replace("_", " ").str.title()

# New column with broader Measures
score_card["Category"] = score_card["Measure"]

score_card["Category"] = score_card["Category"].replace(
    {
        "Increase Peak Person Throughput": "Congestion Mitigation",
        "Reduction In Peak Period Delay": "Congestion Mitigation",
        "Reduction In Fatal And Injury Crashes": "Safety",
        "Reduction In Injury Rates": "Safety",
        "Increase Access To Jobs": "Accessibility Increase",
        "Increase Access Jobs To Dac": "Accessibility Increase",
        "Commercial Dev Developed": "Economic Dev.",
        "Tons Of Goods Impacted": "Economic Dev.",
        "Improve Air Quality": "Environment",
        "Impact Natural Resources": "Environment",
        "Support Of Trasnportation": "Land Use",
    }
)

# Get total scores
total = (
    score_card.groupby(["Category"])
    .agg({"Measure Score": "sum"})
    .rename(columns={"Measure Score": "Total Category Score"})
)
score_card = pd.merge(score_card, total, how="left", on=["Category"])

In [20]:
_utils.dual_bar_chart(
    score_card,
    "Category",
    "Category:N",
    "Total Category Score:Q",
    "Measure:N",
    "Measure Score:Q",
)

alt.HConcatChart(...)